<a href="https://colab.research.google.com/github/DilkiIshara/Hate-Speech-Detection-System/blob/main/Hate_Speech_Detection_CLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!nvidia-smi

Sun Sep  1 09:08:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
!pip install tensorflow

In [15]:
!pip install sentence-transformers

In [86]:
import csv
import re
from collections import defaultdict as ddict

import nltk
import numpy as np
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
# from transformers import (AdamW, XLMRobertaForSequenceClassification,
#                           XLMRobertaTokenizer, get_linear_schedule_with_warmup)
from transformers import XLMRobertaTokenizer, XLMRobertaModel, MarianMTModel, MarianTokenizer, AutoTokenizer,AutoModelForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

Load The Data set

In [78]:
englishDataPath = '/content/drive/MyDrive/HSDData/EnglishDataSet.csv'


try:
    english_train_data = pd.read_csv(englishDataPath)
except pd.errors.EmptyDataError:
    print("The file is empty or could not be loaded.")
except pd.errors.ParserError as e:
    print(f"Parser Error: {e}")

Display all the row data

In [79]:
english_train_data

,text,label-word,label
0,It should be legal for a husband to rape his wife,hate,1
1,dalits are lowlife scum,hate,1
2,dalits are lowlives,hate,1
3,It was a better world when women dare not ques...,hate,1
4,I don't work this hard so that those immigrant...,hate,1
...,...,...,...
40618,we should forcibly remove all the foreigners f...,hate,1
40619,we should forcibly remove all the immigrants f...,hate,1
40620,dalits are unworthy of the Gods love,hate,1
40621,dalits are worthless,hate,1


Clean Up the data Set and Display data

In [87]:
def cleaning_text(df, text_field):
  df[text_field] = df[text_field].str.lower().str.replace(r'[^\w\s]', '', regex=True)
  return df

train = cleaning_text(english_train_data, "text")
train

,text,label-word,label
0,it should be legal for a husband to rape his wife,hate,1
1,dalits are lowlife scum,hate,1
2,dalits are lowlives,hate,1
3,it was a better world when women dare not ques...,hate,1
4,i dont work this hard so that those immigrants...,hate,1
...,...,...,...
40618,we should forcibly remove all the foreigners f...,hate,1
40619,we should forcibly remove all the immigrants f...,hate,1
40620,dalits are unworthy of the gods love,hate,1
40621,dalits are worthless,hate,1


Save the clean Up data in train.tsv

In [88]:
output_filename = '/content/drive/MyDrive/HSDData/train.tsv'
train.to_csv(output_filename, sep='\t', index=False, header=True, encoding='utf-8')

Split the Data into training and testing sets and save in separate files

In [89]:
trainDataSetFileName = '/content/drive/MyDrive/HSDData/train_DataSet.tsv'
testDataSetFileName = '/content/drive/MyDrive/HSDData/test_DataSet.tsv'

train_DataSet = train.sample(frac=0.8, random_state=73)
test_DataSet = train.drop(train_DataSet.index)


train_DataSet.to_csv(trainDataSetFileName,sep='\t', index=False)
test_DataSet.to_csv(testDataSetFileName, sep='\t',index=False)

In [90]:
label_categories = {
    'nothate':0,
    'hate':1
}

def load_dataset():
  lines = []
  with open(trainDataSetFileName, "r", encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for line in reader:
      lines.append(line)
      # print("Data" + line[0])
    texts=[]
    label =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      text = line[0]
      labelValue = line[1]

      # print("Data" + text)
      # print("Label" + labelValue)

      texts.append(text)
      label.append(label_categories[labelValue])
    label_list = ["hate", "nothate"]
    dic_data['texts']=texts
    dic_data['label']=label
    print(len(texts))
    print(len(label))
    return dic_data,label_list

train_DataList, label_List= load_dataset()

32498
32498


In [93]:
data = pd.DataFrame(
    {
        'EnglishData' : train_DataList['texts'],
        'Label' : train_DataList['label']
    }
)

data


,EnglishData,Label
0,0ld p3opl3 are trsh,1
1,coffee is fucking disgusting,0
2,faggots are great at being beastly,1
3,i dont want to sound racist or anything but as...,1
4,fuck seaworld orcas dont belong in captivity,0
...,...,...
32493,britain has has become much more diverse and e...,0
32494,i feel like as a dyke i should be offended but...,0
32495,so pleased my son isnt a spazz,1
32496,hindus hate all other religions,1


In [92]:
# Load tokenizer and model for XLM-RoBERTa
xlm_roberta_model_name = "xlm-roberta-base"
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained(xlm_roberta_model_name)
xlm_roberta_model = XLMRobertaModel.from_pretrained(xlm_roberta_model_name)

# Load tokenizer and model for translation (English to Sinhala)
translation_model_name = "keshan/SinhalaBERTo"
translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSequenceClassification.from_pretrained(translation_model_name,num_labels=2)

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at keshan/SinhalaBERTo and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Function to compute embeddings
def get_embedding(text, tokenizer, model):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    last_hidden_state = output.last_hidden_state
    embedding = torch.mean(last_hidden_state, dim=1).squeeze()
    return embedding.numpy()

# Compute embeddings for English text
df['english_embedding'] = data['EnglishData'].apply(lambda x: get_embedding(x, xlm_roberta_tokenizer, xlm_roberta_model))

In [ ]:
# Function to translate text
def translate_to_sinhala(texts, tokenizer, model):
    translated_texts = []
    for text in texts:
        translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
        translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
        translated_texts.append(translated_text)
    return translated_texts

/////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

texts = data['Data'].tolist()
labels = data['Label'].tolist()

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

labels = np.array(labels)

model_1 = Sequential()
model_1.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model_1.add(LSTM(units=256, dropout=0.2, recurrent_dropout=0.2))
model_1.add(Dense(units=1, activation='sigmoid'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)
# Print the model architecture summary
model_1.summary()

model_2 = Sequential()
model_2.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model_2.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_2.add(Dense(units=1, activation='sigmoid'))
model_2.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))  # Additional LSTM layer
model_2.add(Dense(units=1, activation='sigmoid'))
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)
model_2.summary()


model_3 = Sequential()
model_3.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model_3.add(LSTM(units=256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_3.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))  # Additional LSTM layer
model_3.add(Dense(units=1, activation='sigmoid'))
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_3.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)
model_3.summary()


model_4 = Sequential()
model_4.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length)) # Add an Embedding layer
model_4.add(Conv1D(filters=64, kernel_size=5, activation='relu')) # Add a Conv1D layer
model_4.add(MaxPooling1D(pool_size=2))
model_4.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)) # Add an LSTM layer
model_4.add(Dense(units=64, activation='relu')) # Add Dense layers
model_4.add(Dense(units=1, activation='sigmoid'))
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_4.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)
model_4.summary()


model_5 = Sequential()
model_5.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length)) # Add an Embedding layer
model_5.add(Conv1D(filters=64, kernel_size=5, activation='relu')) # Add a Conv1D layer
model_5.add(MaxPooling1D(pool_size=2))
model_5.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model_5.add(Dense(units=1, activation='sigmoid'))
model_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_5.fit(padded_sequences, labels, epochs=2, batch_size=64, validation_split=0.2)
model_5.summary()



Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 134ms/step - accuracy: 0.5725 - loss: 0.6796 - val_accuracy: 0.5925 - val_loss: 0.6599
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 180ms/step - accuracy: 0.6940 - loss: 0.6050 - val_accuracy: 0.6700 - val_loss: 0.6623
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.8902 - loss: 0.4120 - val_accuracy: 0.7300 - val_loss: 0.5776
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9593 - loss: 0.1443 - val_accuracy: 0.7325 - val_loss: 0.6726
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.9903 - loss: 0.0376 - val_accuracy: 0.7125 - val_loss: 0.7141


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)             │ (64, 92, 128)               │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (64, 256)                   │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (64, 1)                     │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)             │ (64, 92, 128)               │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (64, 88, 64)                │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (64, 44, 64)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_29 (LSTM)                       │ (64, 128)                   │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (64, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,259,909 (16.25 MB)

 Trainable params: 1,419,969 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,839,940 (10.83 MB)

In [ ]:
label_categories = {
    'NOT':0,
    'OFF':1
}

testDataSetFileName = '/content/drive/MyDrive/HSDData/test_DataSet.tsv'

def load_dataset_test():
  lines = []
  with open(testDataSetFileName, "r", encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for line in reader:
      lines.append(line)
    texts=[]
    label =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      textValue = line[1]
      labelValue = line[4]

      texts.append(textValue)
      label.append(label_categories[labelValue])
    label_list1 = ["OFF", "NOT"]
    dic_data['texts']=texts
    dic_data['label']=label
    return dic_data,label_list1
test_d, label_list1= load_dataset_test()

/////upsdatevvv

In [ ]:
data_test = pd.DataFrame(
    {
        'Data' : test_d['texts'],
        'Label' : test_d['label']
    }
)
test_texts = data_test['Data'].tolist()
test_labels = data_test['Label'].tolist()
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)
test_labels = np.array(test_labels)

from sklearn.metrics import accuracy_score, f1_score
# Evaluate the model_1 on the test set
test_predictions = model_1.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'model_1 Testing Accuracy: {test_accuracy:.4f}')
print(f'model_1 Testing F1 Score: {test_f1:.4f}')

# Evaluate the model_2 on the test set
test_predictions = model_2.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'model_2 Testing Accuracy: {test_accuracy:.4f}')
print(f'model_2 Testing F1 Score: {test_f1:.4f}')

# Evaluate the model_3 on the test set
test_predictions = model_3.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'model_3 Testing Accuracy: {test_accuracy:.4f}')
print(f'model_3 Testing F1 Score: {test_f1:.4f}')

# Evaluate the model_4 on the test set
test_predictions = model_4.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'model_4 Testing Accuracy: {test_accuracy:.4f}')
print(f'model_4 Testing F1 Score: {test_f1:.4f}')

# Evaluate the model_5 on the test set
test_predictions = model_5.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'model_5 Testing Accuracy: {test_accuracy:.4f}')
print(f'model_5 Testing F1 Score: {test_f1:.4f}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
model_1 Testing Accuracy: 0.7020
model_1 Testing F1 Score: 0.6284
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step
model_2 Testing Accuracy: 0.6920
model_2 Testing F1 Score: 0.4797
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step
model_3 Testing Accuracy: 0.6940
model_3 Testing F1 Score: 0.6127
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
model_4 Testing Accuracy: 0.7140
model_4 Testing F1 Score: 0.5903
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
model_5 Testing Accuracy: 0.6800
model_5 Testing F1 Score: 0.4366


////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////

In [ ]:
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

# model = Sequential()
# # Add an Embedding layer
# model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
# # Add a Conv1D layer
# model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# # Add an LSTM layer
# model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
# # Add Dense layers
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)
# model.summary()

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.5982 - loss: 0.6822 - val_accuracy: 0.5850 - val_loss: 0.6729
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.5922 - loss: 0.6525 - val_accuracy: 0.6850 - val_loss: 0.6129
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9212 - loss: 0.3371 - val_accuracy: 0.7250 - val_loss: 0.6619
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.9825 - loss: 0.0488 - val_accuracy: 0.7275 - val_loss: 0.9600
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.9995 - loss: 0.0078 - val_accuracy: 0.7225 - val_loss: 1.2223


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)             │ (64, 92, 128)               │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (64, 88, 64)                │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (64, 44, 64)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_22 (LSTM)                       │ (64, 128)                   │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (64, 64)                    │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (64, 1)                     │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,284,485 (16.34 MB)

 Trainable params: 1,428,161 (5.45 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,856,324 (10.90 MB)

Highest accuracy

In [ ]:
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

# model = Sequential()

# # Add an Embedding layer
# model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))

# # Add a Conv1D layer
# model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

# model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))

# model.add(Dense(units=1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.summary()

# model.fit(padded_sequences, labels, epochs=2, batch_size=64, validation_split=0.2)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.5611 - loss: 0.6824 - val_accuracy: 0.5850 - val_loss: 0.6765
Epoch 2/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.6680 - loss: 0.6458 - val_accuracy: 0.7100 - val_loss: 0.5937


In [ ]:
import torch
!pip install transformers
!pip install tokenizers
!pip install datasets
from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline

! pip install sentencepiece

tokenizer = AutoTokenizer.from_pretrained("keshan/SinhalaBERTo")
model = AutoModelForSequenceClassification.from_pretrained("keshan/SinhalaBERTo",num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at keshan/SinhalaBERTo and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['X']
        label = self.data.iloc[index]['Y']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }

In [ ]:
df = pd.DataFrame(data)
df['max_token_count'] = df['Data'].apply(lambda x: len(tokenizer.encode(x)))

# Find the maximum token count across all rows
max_tokens = df['max_token_count'].max()

print("Maximum token count:", max_tokens)
#=========================================================================================================

dataset = Dataset(data, tokenizer, max_length= max_tokens)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained model
model.to(device)

Maximum token count: 435


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
pip install torchviz